In [4]:
from pymongo import MongoClient
from datetime import datetime, timezone
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv() 
# --- CONFIG ---
MONGO_URI = os.getenv("MONGO_URI")

DB_NAME = "knowledge_graph"
wallet_address = "0xc574ee32e5dadac41a2d03f8b6498661d6dac3a2"

# --- KẾT NỐI DB ---
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

# --- LẤY BORROW TẤT CẢ LENDING POOL ---
borrow_cursor = db["borrows"].aggregate([
    {"$match": {"walletAddress": wallet_address}},
    {"$group": {"_id": None, "total": {"$sum": "$totalAmountOfBorrowInUSD"}}}
])
borrow_result = next(borrow_cursor, {})
borrow_amount = borrow_result.get("total", 0)
print(f"📊 Tổng dư nợ tối đa trên tất cả lending pool (USD): {borrow_amount:.4f}")

# --- LẤY REPAY TẤT CẢ LENDING POOL ---
repay_cursor = db["repays"].aggregate([
    {"$match": {"walletAddress": wallet_address}},
    {"$group": {"_id": None, "total": {"$sum": "$totalAmountOfRepayInUSD"}}}
])
repay_result = next(repay_cursor, {})
repay_amount = repay_result.get("total", 0)
print(f"📊 Tổng số tiền đã trả trên tất cả lending pool (USD): {repay_amount:.4f}")

# --- TÍNH NỢ HIỆN TẠI ---
current_debt = max(borrow_amount - repay_amount, 0)

# --- TÍNH TRUNG BÌNH TÀI SẢN 3 NGÀY ---
wallet_docs = list(db["wallets"].find({
    "address": wallet_address
}))
multichain_wallet_docs = list(db["multichain_wallets"].find({
    "address": wallet_address
}))

now_ts = datetime.now(timezone.utc).timestamp()
three_days_ago = now_ts - 3 * 86400

# Tổng tài sản trung bình
total_avg_balance_all_chains = 0
multi_total_avg_balance_all_chains = 0

# Xử lý ví thường
for i, doc in enumerate(wallet_docs):
    balance_logs = doc.get("balanceChangeLogs", {})
    recent_values = []

    for ts, value in balance_logs.items():
        try:
            ts_int = int(ts)
            if ts_int >= three_days_ago:
                recent_values.append((ts_int, float(value)))
        except:
            continue

    recent_values.sort(reverse=True)
    latest_3 = [val for _, val in recent_values[:3]]
    avg_balance = sum(latest_3) / 3 if latest_3 else 0
    print(f"📄 Wallet thường {i+1}: Trung bình 3 ngày = {avg_balance:.6f}")
    total_avg_balance_all_chains += avg_balance

# Xử lý ví multichain
for i, doc in enumerate(multichain_wallet_docs):
    balance_logs = doc.get("balanceChangeLogs", {})
    recent_values = []

    for ts, value in balance_logs.items():
        try:
            ts_int = int(ts)
            if ts_int >= three_days_ago:
                recent_values.append((ts_int, float(value)))
        except:
            continue

    recent_values.sort(reverse=True)
    latest_3 = [val for _, val in recent_values[:3]]
    avg_balance = sum(latest_3) / 3 if latest_3 else 0
    print(f"📄 Wallet multichain {i+1}: Trung bình 3 ngày = {avg_balance:.6f}")
    multi_total_avg_balance_all_chains += avg_balance

# Tổng tài sản trung bình
total_assets = total_avg_balance_all_chains + multi_total_avg_balance_all_chains
print(f"\n💰 Tài sản trung bình (USD) 3 ngày gần nhất: {total_assets:.6f}")

# --- TÍNH TỶ LỆ NỢ TRÊN TÀI SẢN ---
U = current_debt / (total_assets + 1e-6)
D = 0 if U > 1 else 100 * (1 - U)

# --- IN KẾT QUẢ ---
print("\n📊 PHÂN TÍCH AMOUNTS OWED")
print(f"→ Tổng dư nợ tối đa (USD): {current_debt:.4f}")
print(f"→ Tài sản trung bình 3 ngày gần nhất (USD): {total_assets:.4f}")
print(f"→ Tỷ lệ nợ trên tài sản (U): {U:.6f}")
print(f"🎯 Điểm II - AMOUNTS OWED (D): {D:.6f}")


📊 Tổng dư nợ tối đa trên tất cả lending pool (USD): 104031.8580
📊 Tổng số tiền đã trả trên tất cả lending pool (USD): 84436.8220
📄 Wallet thường 1: Trung bình 3 ngày = 0.793431
📄 Wallet thường 2: Trung bình 3 ngày = 78.434926
📄 Wallet thường 3: Trung bình 3 ngày = 16.054059
📄 Wallet thường 4: Trung bình 3 ngày = 0.000000
📄 Wallet thường 5: Trung bình 3 ngày = 0.000000
📄 Wallet thường 6: Trung bình 3 ngày = 0.000000
📄 Wallet thường 7: Trung bình 3 ngày = 0.000000
📄 Wallet multichain 1: Trung bình 3 ngày = 63.248837

💰 Tài sản trung bình (USD) 3 ngày gần nhất: 158.531253

📊 PHÂN TÍCH AMOUNTS OWED
→ Tổng dư nợ tối đa (USD): 19595.0360
→ Tài sản trung bình 3 ngày gần nhất (USD): 158.5313
→ Tỷ lệ nợ trên tài sản (U): 123.603614
🎯 Điểm II - AMOUNTS OWED (D): 0.000000
